This file explores the integration between listenbrainz, musicbrainz, and spotify.

It's separate from the application logic because the approach requires exploration to optimize and understand.
There's a lot more edge cases integratiing multiple edge cases than I had to contend with in using only Spotify.

Below, I sort out those edge cases, explore likelihood of error cases, so I can encode these states in tests.

Import Libs

In [22]:
import os
import time
import json
import requests
from requests.auth import HTTPBasicAuth

Set up Script Variables

In [2]:
artists = ["Noah Gundersen"]

Load Environment Variables

In [3]:
listenbrainz_api_key = os.getenv("LISTENBRAINZ_API_KEY")
spotify_client_secret = os.getenv("SPOTIFY_CLIENT_SECRET")
spotify_client_id = os.getenv("SPOTIFY_CLIENT_ID")
# listenbrainz_api_key, spotify_client_secret, spotify_client_id

Provide useful context around request errors

In [4]:
def handle_error(response):
    try:
        response.raise_for_status()
    except HTTPError as error:
        error.add_note(f"Request: {error.request}")
        error.add_note(f"Request headers: {error.request.headers}")
        error.add_note(f"Request body: {error.request.body}")
        error.add_note(f"Response: {error.response}")
        error.add_note(f"Response headers: {error.response.headers}")
        error.add_note(f"Response body: {error.response.text}")
        raise

Get Spotify Auth Token

In [5]:
spotify_auth = requests.post(
    url="https://accounts.spotify.com/api/token",
    auth=HTTPBasicAuth(spotify_client_id, spotify_client_secret),
    data={"grant_type": "client_credentials"}
)
handle_error(spotify_auth)

auth_json = spotify_auth.json()
spotify_token = auth_json['access_token']
# spotify_token

Get playlist based on artists

In [6]:
lb_radio = requests.get(
    url="https://api.listenbrainz.org/1/explore/lb-radio",
    params={
        "mode": "easy",
        "prompt": " ".join([f'artist:({artist})' for artist in artists])
    },
    headers={"Authorization": f"Bearer {listenbrainz_api_key}"}
)
handle_error(lb_radio)

tracks = lb_radio.json()["payload"]["jspf"]["playlist"]["track"]
len(tracks), tracks[:2]

(36,
 [{'album': 'Love Lost Freedom Found',
   'creator': 'Gary Go feat. Audra Mae',
   'duration': 320000,
   'extension': {'https://musicbrainz.org/doc/jspf#track': {'artist_identifiers': ['388d0edc-7e0c-46a9-855e-436e565fa147',
      '0fe0bb6a-adad-4554-917e-6ef251bbdafc'],
     'release_identifier': 'https://musicbrainz.org/release/b97dd1cc-8683-48ff-b2b3-4b1933128e8e'}},
   'identifier': ['https://musicbrainz.org/recording/ff797336-7b3a-48df-84ac-e994cd413e94'],
   'title': 'Love Lost Freedom Found'},
  {'album': 'I Pledge Allegiance to Myself',
   'creator': 'Lizzie West & The White Buffalo',
   'duration': 165000,
   'extension': {'https://musicbrainz.org/doc/jspf#track': {'artist_identifiers': ['af84d63a-2211-4fc1-b4ec-ce0ffd4c0279',
      'a3f36ee1-c236-49eb-b598-078c788781d8'],
     'release_identifier': 'https://musicbrainz.org/release/1fd60e53-9352-4d2d-a448-7b7f6400b167'}},
   'identifier': ['https://musicbrainz.org/recording/8607da44-c1b1-42c7-a13e-4efd4c777279'],
   'tit

In [7]:
identifiers = []
for track in tracks:
    if len(track['identifier']) > 1:
        raise Exception(f'{track["title"]} by {track["creator"]} has multiple identifiers: {", ".join(track["identifiers"])}')
    identifiers += track["identifier"]
mbids = [x.split("/")[-1] for x in identifiers]
identifiers[:2], mbids[:2]

(['https://musicbrainz.org/recording/ff797336-7b3a-48df-84ac-e994cd413e94',
  'https://musicbrainz.org/recording/8607da44-c1b1-42c7-a13e-4efd4c777279'],
 ['ff797336-7b3a-48df-84ac-e994cd413e94',
  '8607da44-c1b1-42c7-a13e-4efd4c777279'])

In [27]:
all_relations = []
all_isrcs = []
spotify_tracks = []
rate_limit_remaining = 900  # Arbitrary > 0 number
rate_limit_max = 1000 # Slightly larger arbitrary > 0 number
for mbid in mbids:
    time.sleep((rate_limit_max - rate_limit_remaining) / 1000)
    
    # https://musicbrainz.org/doc/MusicBrainz_API#Lookups
    mb_recording = requests.get(
        url=f"https://musicbrainz.org/ws/2/recording/{mbid}",
        params={"fmt": "json", "inc": "isrcs url-rels artists"},
        headers={"User-Agent": "mixtapestudy.com/0.0 ( douglas@builtonbits.com )"}
    )
    handle_error(mb_recording)
    
    recording_json = mb_recording.json()
    rate_limit_remaining = int(mb_recording.headers["X-RateLimit-Remaining"])
    rate_limit_max = int(mb_recording.headers["X-RateLimit-Limit"])
    relations = recording_json["relations"]
    isrcs = recording_json["isrcs"]
    artist_credit = recording_json["artist-credit"]

    if isrcs:
        query_string = f"isrc:{isrcs[0]}"
    elif relations:
        raise Exception(f"Finally found a relation: {relations}, this never happened before")
    else:
        query_string = f'track:{recording_json["title"]}'
        if artist_credit:
            query_string += " " + " ".join([f'artist:{artist["artist"]["name"]}' for artist in artist_credit])
        
    print(f"query_string: {query_string}")
    
    # https://developer.spotify.com/documentation/web-api/reference/search
    spotify_search = requests.get(
        url="https://api.spotify.com/v1/search",
        params={"type": "track", "q": query_string},
        headers={"Authorization": f"Bearer {spotify_token}"}
    )
    handle_error(spotify_search)

    spotify_json = spotify_search.json()
    # print([t["id"] for t in spotify_json["tracks"]["items"]])
    if spotify_json["tracks"] and spotify_json["tracks"]["items"]:
        spotify_tracks.append(spotify_json["tracks"]["items"][0])
    
    all_relations += relations
    all_isrcs += isrcs
    
    # break

(len(all_relations), len(all_isrcs), len(spotify_tracks)), all_relations[:2], all_isrcs[:2]

query_string: track:Love Lost Freedom Found artist:Gary Go artist:Audra Mae
query_string: track:I Pledge Allegiance to Myself artist:Lizzie West artist:The White Buffalo
query_string: track:Someday Never Comes artist:Billy Valentine artist:The Forest Rangers
query_string: track:Shine (Axmod remix) artist:Benjamin Francis Leftwich
query_string: track:God Don’T Talk To Strangers artist:Noah Gundersen
query_string: track:Damn, Sam artist:Ryan Adams
query_string: track:Every Age artist:José González artist:The String Theory
query_string: track:Forever Young artist:Audra Mae artist:The Forest Rangers
query_string: track:Heart Attack artist:The White Buffalo
query_string: track:Come On Back (feat. Battleme) artist:The Forest Rangers
query_string: track:Every Time I See a Bird artist:Benjamin Francis Leftwich
query_string: track:The Future artist:Noah Gundersen
query_string: track:Set Two Intro artist:Ryan Adams
query_string: track:El Invento (Dub Version) artist:José González
query_string: t

((0, 8, 31), [], ['GB5UQ1900280', 'AULI01500610'])

In [28]:
[
    (st["id"], st["uri"], st["name"], 
    [a["name"] for a in st["artists"]])
    for st in spotify_tracks
]

[('5R7B0yMEekE25qxYUkOuLA',
  'spotify:track:5R7B0yMEekE25qxYUkOuLA',
  'Love Lost Freedom Found',
  ['Gary Go', 'Audra Mae']),
 ('5k6mgVoma3ylHtr4tit0s2',
  'spotify:track:5k6mgVoma3ylHtr4tit0s2',
  'Rope Me In And Smoke Me',
  ['Lizzie West', 'White Buffalo']),
 ('52Xja4luAi0O9yJ1RQeaqe',
  'spotify:track:52Xja4luAi0O9yJ1RQeaqe',
  'Someday Never Comes',
  ['Billy Valentine', 'The Forest Rangers']),
 ('1WpnRu9iSDDR4F7LCNhaIW',
  'spotify:track:1WpnRu9iSDDR4F7LCNhaIW',
  'GOD DON’T TALK TO STRANGERS',
  ['Noah Gundersen']),
 ('07A4I35sBKxJ43iPZmWMqZ',
  'spotify:track:07A4I35sBKxJ43iPZmWMqZ',
  'Damn Sam (I Love a Woman That Rains)',
  ['Ryan Adams']),
 ('79BFr9vsWHtZHLHnvhajEb',
  'spotify:track:79BFr9vsWHtZHLHnvhajEb',
  'Every Age - Live',
  ['José González', 'The String Theory']),
 ('5kjxTu3UD2X5JNs7D5T1cf',
  'spotify:track:5kjxTu3UD2X5JNs7D5T1cf',
  'Cub - Live At Shepherd’s Bush Empire',
  ['Roo Panes']),
 ('2yLK0rMbzADBghilbqcGH1',
  'spotify:track:2yLK0rMbzADBghilbqcGH1',
  '